In [28]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from sklearn import metrics 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import  confusion_matrix
from sklearn.feature_selection import VarianceThreshold
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_excel(r"C:\Users\RameshMisale\Documents\Profile_data.xlsx")

In [3]:
df1 = df

In [4]:
df1['Resubmit'] = pd.to_datetime(df1['Resubmit'])
df1['Resubmit'] = df1['Resubmit'].notnull().astype(int)

In [5]:
columns_to_fill = ['rcra_non_haz_exempt','halogens_flag','no_reactivity_flag','layered','viscosity','odor_flag','ph_flag',
    'flash_point_flag','boiling_point_flag','btu_per_lbs','pumpable_waste_flag','polymerizable_flag','benzene_waste_flag',
    'voc_100_ppm','marine_pollutant_flag','origin_code','sds_attached','specific_gravity','benzene_section_flag',
    'max_benzene_flag','benzene_water','prohibited_land_disposal','uts_waste','voc_500_ppm','specialpricing_flag',
    'intercompany_flag','mgp_flag','pa_waste_catogory','debris','compressed_gas','analytical_ind',
    'generatorknowledge_ind','sds_ind','formulary_attached','analytical_attached','sample_provided','mgplock_flag',
    'naics_flag','federal_universal_waste','generator_state_universal_waste']
df1[columns_to_fill] = df1[columns_to_fill].fillna(0, inplace=False)

In [6]:
col = ['water_percentage', 'toc_percentage'] #filling with mean for these 2 columns
df1[col] = df1[col].fillna(df1[col].mean(), inplace=False)
#df1.isnull().sum()

In [7]:
object_columns = df1.select_dtypes(include=['object','datetime64']).columns
df_dropped_objects = df1.drop(object_columns, axis=1)
# df_dropped_objects.dtypes

In [8]:
#It will remove the zero variance features

var_thres=VarianceThreshold(threshold=0)
var_thres.fit(df_dropped_objects)

C:\Users\RameshMisale\anaconda3\Lib\site-packages\sklearn\feature_selection\_variance_threshold.py:111: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)
C:\Users\RameshMisale\anaconda3\Lib\site-packages\sklearn\feature_selection\_variance_threshold.py:119: RuntimeWarning: All-NaN slice encountered
  self.variances_ = np.nanmin(compare_arr, axis=0)


VarianceThreshold(threshold=0)

In [9]:
constant_columns = [column for column in df_dropped_objects.columns
                    if column not in df_dropped_objects.columns[var_thres.get_support()]]
print(f"Number of constant columns: {len(constant_columns)}")
for feature in constant_columns:
    unique_values = df_dropped_objects[feature].unique()
    print(f"Constant column: {feature}")
    print(f"Unique values: {unique_values}")
    print("\n")


Number of constant columns: 17
Constant column: phenolics_ppm
Unique values: [nan  0.]


Constant column: cylinder_flag
Unique values: [nan  0.]


Constant column: minimum_packaging_requirements
Unique values: [nan]


Constant column: outbound_profile_taxes
Unique values: [nan]


Constant column: inbound_oubound_id_xref
Unique values: [nan]


Constant column: ky_report_physical_state_ind
Unique values: [nan]


Constant column: ky_report_onsite_ind
Unique values: [nan]


Constant column: contract_id
Unique values: [nan]


Constant column: sic_code
Unique values: [nan]


Constant column: pet_chem_flag
Unique values: [nan  0.]


Constant column: pet_chem_actual
Unique values: [nan]


Constant column: cokeoven_flag
Unique values: [ 0. nan]


Constant column: container_size_flag
Unique values: [ 1. nan]


Constant column: waste_type_id
Unique values: [nan]


Constant column: highly_toxic_flag
Unique values: [nan  0.]


Constant column: incin_prep_flag
Unique values: [nan  1.]


Constant col

In [10]:
data = df_dropped_objects.drop(constant_columns,axis=1) #dropped the columns here 

In [11]:
def correlation(dataset,threshold):
    col_corr=set()
    corr_matrix=dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i,j]>threshold):
                colname=corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [12]:
corr_features=correlation(data,0.8)
len(set(corr_features))

48

In [13]:
data1=data.drop(corr_features,axis=1) # dropped the correlated features here

In [14]:
threshold_percentage = 90 
null_percentage = (data1.isnull().sum() / len(data1)) * 100
columns_to_drop = null_percentage[null_percentage > threshold_percentage].index
data1_dropped = data1.drop(columns=columns_to_drop)
#data1_dropped.isnull().sum()

In [15]:
data1_dropped = data1_dropped.fillna(0) #filling the null values with 0
#data1_dropped.isnull().sum() 

In [16]:
dff = data1_dropped.drop(columns=['ReturnCount','DaysAssignReadyForGenSign','profile_id','web_profile_number',
                                  'DaysSubmitToAssign','DaysAssignReadyForGenSign','DaysReadyForGenSignSentForGenSign',
                                  'DaysDocSignReturnedToApproved','container_type_id','DaysInitiatedToSubmitted',
'DaysSentForGenSignToDocSignReturned','vendor_id','ContractID','ldr_class_id','CustomerId','CollectionId',
'HCSId','Recert','is_template_profile_flag','AssignUser_id','status_code_id','source_code_id','form_code_id',
'management_method_code_id','outbound_profile_id','price_type_code_id','parent_profile_id','health_chemical_identity_id',
'flammability_chemical_identity_id','reactivity_chemical_identity_id','process_code_id',
'SalesrepID','InternalCoordinatorID','MarketDriverID','InsideSalesRepID','requested_process_code_id'],axis=1)
dff.shape

(59378, 84)

In [17]:
vif_data = pd.DataFrame()
vif_data["feature"] = dff.columns
vif_data["VIF"] = [variance_inflation_factor(dff.values, i) for i in range(dff.shape[1])]
print(vif_data)

                            feature        VIF
0                             IsHaz  41.992835
1                          Resubmit   1.245218
2              outbound_profile_ind   1.218593
3                     isRecertified   1.227366
4                     standard_flag  32.184140
..                              ...        ...
79               chem_coke_pet_flag   1.282394
80                       label_type   2.613335
81          federal_universal_waste   1.663563
82  generator_state_universal_waste   1.115983
83                     revision_num   1.885647

[84 rows x 2 columns]


In [18]:
high_vif_columns = vif_data[vif_data["VIF"] > 5]["feature"].tolist()
high_vif_columns

['IsHaz',
 'standard_flag',
 'urgent_flag',
 'toc_percentage',
 'generatorknowledge_ind']

In [19]:
df_final = dff.drop(columns=high_vif_columns)
df_final.shape

(59378, 79)

In [22]:
XX = df_final.drop('Resubmit', axis='columns')
yy= df_final['Resubmit']
XX_train, XX_test, yy_train, yy_test = train_test_split(XX,yy, test_size=0.2,random_state=42,stratify=yy)

In [24]:
count_0_class,count_1_class = df_final.Resubmit.value_counts()
df_class_0 = df_final[df_final["Resubmit"]==0]
df_class_1 = df_final[df_final["Resubmit"]==1]
X_train, X_test, y_train, y_test = train_test_split(XX,yy, test_size=0.2,random_state=15,stratify=yy)

In [25]:
df_class_0_under = df_class_0.sample(count_1_class)
df_test_under = pd.concat([df_class_0_under, df_class_1],axis=0)

print('Random Under-sampling:')
print(df_test_under.Resubmit.value_counts())

Random Under-sampling:
0    6757
1    6757
Name: Resubmit, dtype: int64


In [26]:
XX = df_test_under.drop('Resubmit', axis='columns')
yy= df_test_under['Resubmit']
XX_train, XX_test, yy_train, yy_test = train_test_split(XX,yy, test_size=0.2,random_state=42,stratify=yy)

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
logistic_regression = LogisticRegression()
logistic_regression.fit(XX_train, yy_train)
y_pred_lr_train = logistic_regression.predict(XX_train)
y_pred_lr_test = logistic_regression.predict(XX_test)

# Accuracy
accuracy_lr_train = accuracy_score(yy_train, y_pred_lr_train)
accuracy_lr_test = accuracy_score(yy_test, y_pred_lr_test)
print(f'Logistic Regression Training Accuracy: {accuracy_lr_train}')
print(f'Logistic Regression Testing Accuracy: {accuracy_lr_test}')

# # classification report
# print(classification_report(yy_test, y_pred_lr_test))
# print(confusion_matrix(yy_test, y_pred_lr_test))

precision_lr_test = precision_score(yy_test, y_pred_lr_test)
recall_lr_test = recall_score(yy_test, y_pred_lr_test)
f1_lr_test = f1_score(yy_test, y_pred_lr_test)

# Display precision, recall, and f1-score
print(f'\nPrecision: {precision_lr_test:.4f}')
print(f'Recall: {recall_lr_test:.4f}')
print(f'F1-Score: {f1_lr_test:.4f}')


Logistic Regression Training Accuracy: 0.6957728239755804
Logistic Regression Testing Accuracy: 0.6888642249352571

Precision: 0.6756
Recall: 0.7261
F1-Score: 0.7000


C:\Users\RameshMisale\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
import pickle
file_path = (r'C:\Users\RameshMisale\Downloads\Harsco_model.pkl')

# Open the file in binary write mode
with open(file_path, 'wb') as file:
    # Use pickle.dump() to serialize and save the data to the file
    pickle.dump(file_path, file)

print(f'Data saved to {file_path}')

Data saved to C:\Users\RameshMisale\Downloads\Harsco_model.pkl
